In [81]:
import torch
from torch import nn
import numpy
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.utils.data import DataLoader
import scipy.io

In [82]:
total = scipy.io.loadmat('sorted_zWF_data.mat')
waves = total['zWF']
sorted = total['sorted']

waves = torch.tensor(waves)
sorted = torch.tensor(sorted)


# Create a list to hold tuples
data = []

# Generate data for each index
for i in range(len(waves)):
    for k in range(len(waves.T)):
        waves[i,k] = float(waves[i,k])
    vector = waves[i,:] # Generating a random vector of length 31
    integer = sorted[:,i]  # Generating a random integer
    data.append(((vector), float(integer)))

split_index = int(len(data) * 0.8)

# Split the variable list
train_data = data[:split_index]
test_data = data[split_index:]


In [83]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #self.flatten = nn.Flatten()
        self.l1 = nn.Linear(len(waves.T), 50)
        self.l2 = nn.Linear(50, 50)
        self.l3 = nn.Linear(50,20)
        self.l4 = nn.Linear(20,3)

    def forward(self, x):
        #x = self.flatten(x)
        x = F.relu(self.l1(x))
        x = F.relu(self.l2(x))
        x = F.relu(self.l3(x))
        output = self.l4(x)
        return output


model = Net()




In [84]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [85]:
#HELP FROM CHATGPT
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        vector, integer = self.data[index]
        return vector, integer

# Assuming you have created train_data and test_data
# Create datasets for both parts
train_dataset = CustomDataset(train_data)
test_dataset = CustomDataset(test_data)

# Create data loaders for both datasets

In [86]:



train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle= True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle= True)

learning_rate = 1e-4
batch_size = 64

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")


Epoch 1
-------------------------------


RuntimeError: mat1 and mat2 must have the same dtype, but got Double and Float

In [ ]:
sample_num = 4 # select a random sample
x = test[sample_num][0]
x = x[None,:,:,:]
with torch.no_grad():
   r = model(x)
print('neural network output pseudo-probabilities:', r)
print('neural network output class number:', torch.argmax(r).item())